In [1]:
import pandas as pd
import numpy as np
import regex

import matplotlib.pyplot as plt

from sklearn.metrics import f1_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler


from itertools import product
from sklearn import model_selection
from sklearn.model_selection import KFold

from tqdm.notebook import tqdm


%matplotlib inline


Сначала мутим словарь

In [75]:
doc_to_title = {}
with open("data/docs_titles/lemm_titles.csv") as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split(',', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title=data[1]
#             title = regex.sub(r'[^\p{Cyrillic}]+', ' ', data[1]).lower()
            title = ' '.join(word for word in title.split() if len(word)>2)
        doc_to_title[doc_id] = title
print (len(doc_to_title))

28026


In [76]:
# doc_to_title

Функция делает словарь {группа: [ (тайтл, таргет) ]}

In [66]:
train_data = pd.read_csv('data/train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

Минимальная длина пересечений 25, значит фич не больше 25 может быть получается

In [67]:
def getTrain(params):
    y_train = []
    X_train = []
    for new_group in tqdm(traingroups_titledata):
        docs = traingroups_titledata[new_group]
        ln = 0
        for j in range(0, len(docs)):
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            ln += len(words_j)
        ln /= len(docs)
        for k, (doc_id, title, target_id) in enumerate(docs):
            y_train.append(target_id)
            all_dist = []  
            words = set(title.strip().split())
            for j in range(0, len(docs)):
                if k == j:
                    continue
                doc_id_j, title_j, target_j = docs[j]
                words_j = set(title_j.strip().split())
                all_dist.append(len(words.intersection(words_j)))
            all_dist.sort(reverse=True)
            all_dist = all_dist[0:params]
            all_dist.append(abs(len(words) - ln))
#             print(abs(len(words) - ln))
            X_train.append(all_dist)
#             X_train.append(abs(len(words) - ln))            
            
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    
    return X_train, y_train

In [68]:
a, b = getTrain(10)
print(a)


[[ 1.08776295  0.92357     1.09527046 ...  0.96980722  1.02092127
  -0.61365744]
 [ 1.56706747  1.99577152  2.22111893 ...  0.96980722  1.02092127
  -0.14775971]
 [ 0.12915391  0.38746923  0.53234623 ...  0.33577968  0.37654085
   0.27654002]
 ...
 [-1.30875964 -1.22083306 -1.15642647 ... -0.93227539 -0.91221997
   1.02973831]
 [ 1.08776295 -0.68473229 -0.59350224 ... -0.29824785 -0.26783956
   0.33034316]
 [-0.82945512 -0.68473229 -0.59350224 ... -0.93227539 -0.91221997
   1.17894261]]


In [69]:
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)

In [70]:
# X_train

In [71]:

splits = 6
kf = KFold(n_splits=splits)

def max_barrier(clf, X, y):
    """
    Подбираем порог линейной модели, 
    по достижению которого относим объект 
    к классу 1
    """
    (max_score, max_barr) = (0, 0)
    for barr in np.arange (0, 1, 0.01):
            probe = list(zip(*clf.predict_proba(X)))[1]
            score = f1_score(y, probe > barr)
#             print(barr, score)
            if score > max_score:
#                 print("AAaaaaaaaaaaaaaaaaa")
#                 print(barr, score)
                max_score, max_barr = score, barr
    return max_score, max_barr

def get_best(penalty_list, c_list, l1_ratio_list, fiches_list):
    """
    Делаем кросс валидацию и, таким образом, 
    подбираем оптимальные параметры.
    """
    max_result = -1
    for i, (penalty, C, l1_ratio, fiches) in tqdm(list(enumerate(product(penalty_list, c_list, l1_ratio_list, fiches_list)))):
        result = 0.0
        X_tran, y_tran = getTrain(fiches)
        for train_index, test_index in kf.split(X_tran):
            X_tr, X_tst = X_tran[train_index], X_tran[test_index]
            y_tr, y_tst = y_tran[train_index], y_tran[test_index]
            
            clf = LogisticRegression(C=C, penalty=penalty, class_weight='balanced', solver='saga', l1_ratio=l1_ratio, max_iter = 1000)
            clf.fit(X_tr, y_tr)
            
            score, barrier = max_barrier(clf, X_tst, y_tst)
#             print(clf.predict(X_tst))
#             print(penalty, C, l1_ratio, barrier)
#             probe = clf.predict_proba(X_tst))
            result += score
        
        result /= splits
        print(result, fiches)
        if result > max_result:
            max_result = result
            params = {'C': C, 'penalty': penalty, 'l1_ratio': l1_ratio, 'barrier':barrier, 'fiches': fiches}
            print(params)
    
    print('max_f1_score =', max_result, 'with', params)
    return params

In [74]:
l1_ratio_list = [0.95]#np.arange(0.1, 1.1, 0.01)
c_list = [1]#np.logspace(10, 1, )
fiches_list = [21]#np.arange(3, 25, 1)

penalty_list = ['elasticnet']

print(c_list)
print(l1_ratio_list)
print(fiches_list)

log_params = get_best(penalty_list, c_list, l1_ratio_list, fiches_list)


[1]
[0.95, 0.5, 0.1]
[21]



0.6935027730909339 21
{'C': 1, 'penalty': 'elasticnet', 'l1_ratio': 0.95, 'barrier': 0.55, 'fiches': 21}



0.69268042308667 21



0.6926202259376347 21

max_f1_score = 0.6935027730909339 with {'C': 1, 'penalty': 'elasticnet', 'l1_ratio': 0.95, 'barrier': 0.55, 'fiches': 21}


In [77]:
X_train, y_train = getTrain(21)

тест надо чуть поменять еще одна фича же

In [78]:
test_data = pd.read_csv('data/test_groups.csv')
testgroups_titledata = {}
for i in range(len(test_data)):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    pair_id = new_doc['pair_id']
    title = doc_to_title[doc_id]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, pair_id, title))
    
X_test = []
pairs_id = []
for new_group in testgroups_titledata:
    docs = testgroups_titledata[new_group]        
    ln = 0
    for j in range(0, len(docs)):
        *_, title_j = docs[j]
        words_j = set(title_j.strip().split())
        ln += len(words_j)
    ln /= len(docs)
    for k, (doc_id, pair_id, title) in enumerate(docs):
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            *_, title_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        
        all_dist.sort(reverse=True)
        all_dist = all_dist[0:log_params['fiches']]
        all_dist.append(abs(len(words) - ln))
        X_test.append(all_dist)
        pairs_id.append(pair_id)
X_test = np.array(X_test)
# X_test = scaler.transform(X_test)
print(X_test.shape)

(16627, 22)


In [79]:
log_params

{'C': 1,
 'penalty': 'elasticnet',
 'l1_ratio': 0.95,
 'barrier': 0.55,
 'fiches': 21}

In [80]:
log_clf = LogisticRegression(C=1, penalty='elasticnet', class_weight='balanced', solver='saga', l1_ratio=0.95, max_iter = 1000)
log_clf.fit(X_train, y_train)

LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=0.95, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='elasticnet',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [81]:
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)

In [82]:
X_test.shape

(16627, 22)

In [83]:
# X_test = scaler.fit_transform(X_test)
probes = list(zip(*log_clf.predict_proba(X_test)))[1]
predict = np.asarray(probes > log_params['barrier'], dtype=int)

In [85]:
test_data["target"] = pd.Series(predict)
with open("predict_stam_1.csv", "w") as f:
    f.write(test_data.to_csv(columns=("pair_id", "target"), index=False))
print("good")

good
